# IBM Applied Data Science Capstone Course by Coursera

# 1.0 Introduction
One issue that people must face in a globalized world is moving far away from home. Whether they are pursuing a dream career, moving closer to family and friends, or just wanting a change of environment, it is inevitable that some people will make big changes in their life and move elsewhere. People would rather move to areas where they feel the most comfortable so they can adjust to their new surroundings a little easier; some may want to move somewhere that has similar food culture, shops, recreational spots, etc. However, when you’re moving somewhere new, it can be overwhelming to search through the whole city and find these areas.

Both New York City (NYC) and Toronto are major metropolitan cities renown for being financial capitals of their respective countries with incredible multiculturalism. However, they are in different countries and have different histories; their environments, though similar in some ways, are vastly different in other ways. Two major boroughs in Toronto and NYC are Downtown Toronto and Manhattan, respectively. When one is moving from Downtown Toronto to Manhattan, it would be ideal to move to a neighbourhood that had the greatest similarity to their home city to make such a big change more comfortable.

This Capstone project aims to analyze neighbourhoods in Toronto and NYC and determine their similarity (this will mostly be based on similarity in venues using Foursquare API), using these insights as a reference for recommending which neighbourhood one should choose in NYC when moving there from Toronto. Following data science methodology, machine learning techniques such as k-means clustering will be used to provide answers to the business question: What similar neighbourhood in NYC would you recommend an individual move to if they were moving from Toronto?


------------------------------------------------------------------------------------------------------------------------------------

# 2.0 Data
The following data will be needed to answer the business question:
-  List of neighbourhoods in Downtown Toronto and their latitude and longitude coordinates
-  List of neighbourhoods in Manhattan and their latitude and longitude coordinates
-  Information of the venues in each of the neighbourhoods

The general breakdown of the data is as follows:
-  Build a dataframe of neighbourhoods and boroughs in Toronto by webscraping the data from Wikipedia
-  Build a dataframe of neighbourhoods and boroughs in NYC by parsing through the JSON file
-  Get the latitude and longitude coordinates of each neighbourhood through geocoder
-  Sort the dataframe so that only Downtown Toronto neighbourhoods are shown
-  Sort the dataframe so that only Toronto neighbourhoods are shown
-  Obtain the venue data for the neighbourhoods using FourSquare API

## 2.1 Toronto Neighbourhood Data

To get the neighbourhood data for Downtown Toronto, we need to webscrape the Wikipedia table that contains Canadian postal codes, boroughs, and neighbourhoods: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Coordinates for each neighbourhood is required to find the nearby venues since we will be using the Foursquare API. 

In [2]:
import pandas as pd # library for data analysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import numpy as np # library for handling data in a vectorized manner

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
!pip install bs4
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### Collecting Toronto Neighbourhood Data

Now that we've imported all the required libraries, we can scrape the Wikipedia page using a BeautifulSoup object.

In [3]:
#url of Wikipedia showing Toronto neighbourhoods
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# send the GET request
req = requests.get(url)

# create the BeautifulSoup object
soup = BeautifulSoup(req.content, 'html.parser')

In [4]:
# With the BeautifulSoup object, scrape the webpage and create a dataframe
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned': # ignore cells with a borough that is Not assigned.
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
# print(table_contents)
Candf = pd.DataFrame(table_contents)
Candf['Borough'] = Candf['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
Candf.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


From this, we can see that we successfully scraped the Wikipedia table and created a dataframe with the postal codes, boroughs, and neighbourhoods of Toronto.

What we need to do next is pass each address through Google's ARCGIS package to get the latitude and longitude coordinates of each neighbourhood (which, again, are needed later on when we use the Foursquare API!).

In [6]:
latitude=[]
longitude=[]
for code in Candf['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

M3A [43.75245000000007, -79.32990999999998]
M4A [43.73057000000006, -79.31305999999995]
M5A [43.65512000000007, -79.36263999999994]
M6A [43.72327000000007, -79.45041999999995]
M7A [43.66253000000006, -79.39187999999996]
M9A [43.662630000000036, -79.52830999999998]
M1B [43.811390000000074, -79.19661999999994]
M3B [43.74923000000007, -79.36185999999998]
M4B [43.70718000000005, -79.31191999999999]
M5B [43.65739000000008, -79.37803999999994]
M6B [43.70687000000004, -79.44811999999996]
M9B [43.65034000000003, -79.55361999999997]
M1C [43.78574000000003, -79.15874999999994]
M3C [43.72168000000005, -79.34351999999996]
M4C [43.68970000000007, -79.30681999999996]
M5C [43.65215000000006, -79.37586999999996]
M6C [43.69211000000007, -79.43035999999995]
M9C [43.64857000000006, -79.57824999999997]
M1E [43.765750000000025, -79.17469999999997]
M4E [43.67709000000008, -79.29546999999997]
M5E [43.64536000000004, -79.37305999999995]
M6E [43.68784000000005, -79.45045999999996]
M1G [43.76812000000007, -79.2

Now that we have the coordinates, we can append them to the original dataframe.

In [7]:
# append the coordinate lists to the dataframe
Candf['Latitude'] = latitude
Candf['Longitude'] = longitude
Candf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


#### Collecting Downtown Toronto Data

The dataframe is all done, now we can filter through it and get the Downtown Toronto borough subset into its own dataframe.

In [9]:
# get the subset of Downtown Toronto
tor_df = Candf[Candf.Borough == "Downtown Toronto"].reset_index(drop=True)
tor_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
4,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493


In [10]:
size = tor_df.shape
print("The Downtown Toronto dataframe has {} rows and {} columns".format(size[0],size[1]))

The Downtown Toronto dataframe has 17 rows and 5 columns


Finally, we need to check for any missing values since we will be merging the dataframes of Downtown Toronto and Manhattan later.

In [11]:
tor_df.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64

In [12]:
tor_df.duplicated().sum()

0

Looks good! Let's move on.

### 2.2 Manhattan Neighbourhood Data

To get Manhattan neighbourhood data, we'll first need to collect New York City data. Luckily, this data can be found here https://geo.nyu.edu/catalog/nyu_2451_34572

We can download the JSON file and open it in this notebook.

After that, we can open the file and take a look at what is inside:

In [13]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [14]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Okay, we can see that this is a huge collection of dictionaries. All the relevant data we need seem to be in the "features" key (in the form of a list of lot of nested dictionaries!), so so let's define a new variable that we can store this data in.

In [15]:
neighborhoods_data = newyork_data['features']

Now we can go even deeper and get a general idea of what is inside this list:

In [16]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Seems like each nested dictionary within this list gives us everything we need! 

Now that we have a list of nested Python dictionaries containing the neighbourhood name, boroughs, and latitude and longitude coordinates, we can convert the necessary information into a pandas dataframe. We can start by creating an empty dataframe with the column headings we need:

In [18]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
NYneighborhoods = pd.DataFrame(columns=column_names)

Now we can loop through the dictionaries and append them to the new dataframe:

In [19]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NYneighborhoods = NYneighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

# check the resulting data
NYneighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


As we did this with the Toronto dataset, let's filter through the NYC data and isolate the Manhattan borough subset.

In [20]:
man_df = NYneighborhoods[NYneighborhoods.Borough == "Manhattan"].reset_index(drop=True)
man_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [21]:
man_df.shape

(40, 4)

Now that we have what we need, we need to check for any missing values or duplicates.

In [22]:
print(man_df.isnull().sum())
print(man_df.duplicated().sum())

Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64
0


Everything checks out! Let's move on.

### 2.3 Merging Both Downtown Toronto and Manhattan Datasets

We will be clustering both Downtown Toronto and Manhattan neighbourhoods, so we will need to merge the two sets together to get a new working dataset.

In [23]:
tor_man = pd.concat([tor_df, man_df], sort=False).reset_index(drop=True)
tor_man.drop(["PostalCode"], axis=1, inplace=True) # drop this column as it is not needed
tor_man.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,Downtown Toronto,St. James Town,43.65215,-79.37587
3,Downtown Toronto,Berczy Park,43.64536,-79.37306
4,Downtown Toronto,Central Bay Street,43.65609,-79.38493


In [24]:
# check the last 5 rows to confirm dataframes were merged
tor_man.tail()

,Borough,Neighborhood,Latitude,Longitude
52,Manhattan,Turtle Bay,40.752042,-73.967708
53,Manhattan,Tudor City,40.746917,-73.971219
54,Manhattan,Stuyvesant Town,40.731000,-73.974052
55,Manhattan,Flatiron,40.739673,-73.990947
56,Manhattan,Hudson Yards,40.756658,-74.000111


In [25]:
tor_man.shape

(57, 4)

We can check if there are any missing or duplicated values, as well as the general info about the dataframe.

In [26]:
print(tor_man.isnull().sum())
print(tor_man.duplicated().sum())

Borough         0
Neighborhood    0
Latitude        0
Longitude       0
dtype: int64
0


In [27]:
tor_man.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Borough       57 non-null     object 
 1   Neighborhood  57 non-null     object 
 2   Latitude      57 non-null     float64
 3   Longitude     57 non-null     float64
dtypes: float64(2), object(2)
memory usage: 1.9+ KB


### 2.4 Foursquare API Data

Now that we have the data for both Downtown Toronto and Manhattan, we can use the Foursquare API to explore all the nearby venues within 1km of each borough. First, we need to define our user credentials for the API, as well as the radius and LIMIT.

In [28]:
# user credentials for foursquare
CLIENT_ID = 'ZDJ2LRI4TYDNXMAEZUDHJSGFHIH203BLERWQN4PGIC4URBSY'
CLIENT_SECRET = 'J2KHNKYDUUFMP3L1PBCG2VUSW5511HAOJUKWC3TNKVSNIXPV'
VERSION = '20180605' # Foursquare API version

radius = 1000
LIMIT = 100 # a default Foursquare API limit value

We can define a function that takes the name of a neighbourhood and its latitude and longitude coordinates, and uses the Foursquare API to find and return all the nearby venues.

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    # create an empty list to store all the results
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
torman_venues = getNearbyVenues(names = tor_man['Neighborhood'],
                                   latitudes = tor_man['Latitude'],
                                   longitudes = tor_man['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tud

Let's check the size of the dataframe and its first few rows:

In [32]:
print(torman_venues.shape)
torman_venues.head()

(5528, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65512,-79.36264,Rooster Coffee,43.651900,-79.365609,Coffee Shop
3,"Regent Park, Harbourfront",43.65512,-79.36264,Sumach Espresso,43.658135,-79.359515,Coffee Shop
4,"Regent Park, Harbourfront",43.65512,-79.36264,Impact Kitchen,43.656369,-79.356980,Restaurant


---------------------------------------------------------------------------------------------------------------------

# 3.0 Methodology

The general breakdown of the methodology is as follows:
-  Explore and organize the venues found by Foursquare API
-  Clustering the neighbourhoods using k-means clustering
-  Visualize the clusters using folium maps
-  Select the neighbourhood that has the most similarity

We can look at how many venues were returned for each neighbourhood (reminder that we set the limit to 100 venues maximum per neighbourhood).

In [33]:
count = torman_venues.groupby('Neighborhood').count()
count

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
Central Harlem,100,100,100,100,100,100
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Christie,100,100,100,100,100,100


In [40]:
print('There are {} uniques categories.'.format(len(torman_venues['Venue Category'].unique())))

There are 347 uniques categories.


It should be noted that there seem to be some neighbourhoods that have less than 100 venues. Let's take a look.

In [34]:
count[count.Venue != 100]

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Marble Hill,81,81,81,81,81,81
"Regent Park, Harbourfront",97,97,97,97,97,97
Rosedale,13,13,13,13,13,13
"St. James Town, Cabbagetown",37,37,37,37,37,37


Luckily, there aren't that many neighbourhoods that have less than 100 venues, but two of them have really low counts which could slightly skew the clustering. 

To address this, it would be best if we only use the top 10 venues for clustering.

In [35]:
# one hot encoding
torman_onehot = pd.get_dummies(torman_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torman_onehot['Neighborhood'] = torman_venues['Neighborhood'] 

# move neighborhood column to the first column
Neighborhood = torman_onehot.pop('Neighborhood')
torman_onehot.insert(0, 'Neighborhood', Neighborhood)
torman_onehot.head()

torman_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Poutine Place,Pub,Public Art,Puerto Rican Restaurant,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shippi

In [36]:
torman_onehot.shape

(5528, 347)

We can group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [37]:
torman_grouped = torman_onehot.groupby('Neighborhood').mean().reset_index()
torman_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Shop,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Hill,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Poutine Place,Pub,Public Art,Puerto Rican Restaurant,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shippi

We can now sort this all into a dataframe that displays the top 10 venues for each neighborhood. Start with creating a function that will sort venues in descending order:

In [39]:
# sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:] # skips the header row
    row_categories_sorted = row_categories.sort_values(ascending=False) 
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torman_grouped['Neighborhood']

for ind in np.arange(torman_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torman_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Memorial Site,Hotel,Sandwich Place,Plaza,Burger Joint,Gourmet Shop,Gym,Gym / Fitness Center
1,Berczy Park,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Restaurant,Cocktail Bar,Park,Gym,Deli / Bodega,Sushi Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Yoga Studio,Park,Gym / Fitness Center,Dessert Shop,Diner,Spa,French Restaurant,Italian Restaurant
3,Carnegie Hill,Gym / Fitness Center,Café,Bakery,Coffee Shop,Italian Restaurant,Yoga Studio,Wine Shop,French Restaurant,Gym,Pizza Place
4,Central Bay Street,Coffee Shop,Sushi Restaurant,Café,Japanese Restaurant,Restaurant,Clothing Store,Yoga Studio,Falafel Restaurant,Gastropub,Gym


### 3.1 k-Means Clustering

Run k-means to cluster the neighborhood into 5 clusters.

In [41]:
# set number of clusters
kclusters = 5

torman_grouped_clustering = torman_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torman_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 0, 4, 1, 0, 3, 4, 3, 3, 0])

Create a new dataframe that includes the cluster label as well as the top 10 venues for each neighborhood.

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torman_merged = tor_man

In [43]:
# merge toronto_grouped with manhattan_data to add latitude/longitude for each neighborhood
torman_merged = torman_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how ='inner')

In [44]:
torman_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Restaurant,Café,Bakery,Pub,Italian Restaurant,Playground,Grocery Store,Performing Arts Venue,Thai Restaurant
1,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Japanese Restaurant,Coffee Shop,Café,Sushi Restaurant,Gastropub,Italian Restaurant,Restaurant,Plaza,Clothing Store,Cosmetics Shop
2,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Café,Coffee Shop,Restaurant,Japanese Restaurant,Italian Restaurant,Gym,Gastropub,Bakery,Sandwich Place,Seafood Restaurant
3,Downtown Toronto,Berczy Park,43.64536,-79.37306,0,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Restaurant,Cocktail Bar,Park,Gym,Deli / Bodega,Sushi Restaurant
4,Downtown Toronto,Central Bay Street,43.65609,-79.38493,0,Coffee Shop,Sushi Restaurant,Café,Japanese Restaurant,Restaurant,Clothing Store,Yoga Studio,Falafel Restaurant,Gastropub,Gym


### 3.2 Folium Map

#### Toronto Map

Now, we can create a map and plot the clusters of the top 10 venues in each neighbourhood. First, we need to get the geographical coordinates of Toronto using the Nominatim function:

In [45]:
address = "Toronto, ON"

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torman_merged['Latitude'], torman_merged['Longitude'], torman_merged['Neighborhood'], torman_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

From the map above, we can see that there are 4 clusters in Toronto, which are clusters 0, 2, 3, and 4. The biggest cluster being cluster 0 (shown in red).

Let's apply the same process to Manhattan. We need the coordinates of NYC first:

In [47]:
address = "New York City, NY"

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of NYC are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of NYC are 40.7127281, -74.0060152.


In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torman_merged['Latitude'], torman_merged['Longitude'], torman_merged['Neighborhood'], torman_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

From this map, we can see that there are 3 clusters in total in Manhattan, which are cluster 1, 3, and 4. 

### 3.3 Comparing Clusters

As you can see, Manhattan is much bigger than Toronto, and has more venues and neighbourhoods to explore compared to Toronto. It would therefore be appropriate and more convenient to select 5 neighbourhoods that are in the Toronto dataset (representing where the individual is moving from) and see which clusters in Manhattan are the most similar.

We can randomly pick from the Toronto dataset. But first, we will need to isolate the Toronto neighbourhoods from the merged dataset.

In [49]:
# separate the Toronto neighbourhoods in the dataframe
toronto_merged = torman_merged[torman_merged.Borough == "Downtown Toronto"]

# randomly sample rows from this dataset
toronto_merged.sample(5)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,"University of Toronto, Harbord",43.66311,-79.40180,3,Café,Bakery,Sandwich Place,Vegetarian / Vegan Restaurant,Restaurant,Beer Bar,Burger Joint,Grocery Store,Mexican Restaurant,Pizza Place
7,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.64285,-79.38076,0,Café,Coffee Shop,Park,Restaurant,Sushi Restaurant,Hotel,Japanese Restaurant,Cocktail Bar,Gym,Deli / Bodega
11,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.65351,-79.39722,3,Café,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Yoga Studio,Pizza Place,Sandwich Place,Arts & Crafts Store,Bar,Cosmetics Shop
1,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Japanese Restaurant,Coffee Shop,Café,Sushi Restaurant,Gastropub,Italian Restaurant,Restaurant,Plaza,Clothing Store,Cosmetics Shop
8,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.64710,-79.38153,0,Café,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Hotel,Sushi Restaurant,Park,Deli / Bodega,Cocktail Bar


So, our 5 neighbourhoods that we have picked are:
1.  University of Toronto, Harbord
2.  Harbourfront East, Union Station, Toronto Islands (This will be shorted to Harbourfront East)
3.  Kensington Market, Chinatown, Grange Park (This will be shortened to Kensington Market)
4.  Garden District, Ryerson
5.  Toronto Dominion Centre, Design Exchange (This will be shortened to Toronto Dominion Centre)

We can see that three of these neighbourhoods fall into cluster 0, while the other two fall into cluster 3.

Let's see which neighbourhoods in Manhattan also fall into these clusters! But once again, we will need to isolate the Manhattan dataset from the merged dataframe.

In [50]:
# separate the Manhattan dataset
manhattan_merged = torman_merged[torman_merged.Borough == "Manhattan"]

Next, we'll need to determine which neighbourhoods fall into cluster 3.

In [52]:
manhattan_clu3 = manhattan_merged[manhattan_merged["Cluster Labels"] == 3]["Neighborhood"].unique()

print("If you are moving to Manhattan from University of Toronto, Harbord or Kensington Market,\nthe similar neighbourhoods in Manhattan you could choose are:\n")

for neighbourhood in manhattan_clu3:
        print(neighbourhood)

If you are moving to Manhattan from University of Toronto, Harbord or Kensington Market,
the similar neighbourhoods in Manhattan you could choose are:

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
East Village
Lower East Side
Manhattan Valley
Morningside Heights
Stuyvesant Town


There are quite a few options! This is good.

We can check to see what results we get for the last Toronto neighbourhood in cluster 0.

In [59]:
manhattan_clu0 = manhattan_merged[manhattan_merged["Cluster Labels"] == 0]["Neighborhood"].unique()

print("If you are moving to Manhattan from Harbourfront East, Garden District, Ryerson, or Toronto Dominion Centre, \nthe similar neighbourhoods in Manhattan you could choose are:\n")

for neighbourhood in manhattan_clu0:
    print(neighbourhood)

If you are moving to Manhattan from Harbourfront East, Garden District, Ryerson, or Toronto Dominion Centre, 
the similar neighbourhoods in Manhattan you could choose are:



Yikes! It seems that there are no neighbourhoods that fall into the same cluster. If you explore the map of Manhattan, you will see that there are no neighbourhoods that belong to cluster 0. This means that these neighbourhoods have nothing in common with any others in Manhattan.

Now that we have our answers, let's take a closer look at cluster 3 and see what exactly Toronto and Manhattan have in common.

In [54]:
torman_merged.loc[torman_merged['Cluster Labels'] == 3, torman_merged.columns[[1] + list(range(5, torman_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Christie,Korean Restaurant,Coffee Shop,Café,Grocery Store,Japanese Restaurant,Theater,Deli / Bodega,Park,Cocktail Bar,Vegetarian / Vegan Restaurant
10,"University of Toronto, Harbord",Café,Bakery,Sandwich Place,Vegetarian / Vegan Restaurant,Restaurant,Beer Bar,Burger Joint,Grocery Store,Mexican Restaurant,Pizza Place
11,"Kensington Market, Chinatown, Grange Park",Café,Art Gallery,Coffee Shop,Vegetarian / Vegan Restaurant,Yoga Studio,Pizza Place,Sandwich Place,Arts & Crafts Store,Bar,Cosmetics Shop
17,Marble Hill,Park,Bus Station,Mexican Restaurant,Bank,Bakery,Pizza Place,Spanish Restaurant,Sandwich Place,Café,Supermarket
18,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Wine Bar,Sandwich Place,Dessert Shop,Gourmet Shop,Ice Cream Shop,Coffee Shop,Pizza Place
19,Washington Heights,Pizza Place,Café,Mobile Phone Shop,Mexican Restaurant,Bakery,Bar,Bank,Grocery Store,Spanish Restaurant,Park
20,Inwood,Mexican Restaurant,Latin American Restaurant,Lounge,Wine Bar,Café,Bank,Deli / Bodega,Spanish Restaurant,Pizza Place,Bakery
21,Hamilton Heights,Bar,Sandwich Place,Mexican Restaurant,Bank,American Restaurant,Pizza Place,Park,Coffee Shop,Ice Cream Shop,Caribbean Restaurant
22,Manhattanville,Bank,Sandwich Place,Italian Restaurant,Park,Coffee Shop,American Restaurant,Indian Restaurant,Café,Mexican Restaurant,Shoe Store
23,Central Harlem,African Restaurant,Bar,Bank,Pizza Place,Mobile Phone Shop,Southern / Soul Food Restaurant,French Restaurant,Gym,Sandwich Place,Café


# 4.0 Results and Discussion

From the clusters, we can see that Downtown Toronto had 4 clusters in total (0, 2, 3, and 4) while Manhattan only had 3 clusters (0, 1, and 4). 

The biggest cluster in Toronto was cluster 0, which is why randomly selecting values from the dataframe had returned cluster 0 neighbourhoods as the majority. Disappointingly, Manhattan does not have any neighbourhoods that fall in this cluster. This could indicate that Toronto and Manhattan are not that similar despite being major multicultural cities.

However, some neighbourhoods are similar to Manhattan, such as those that fell into cluster 3. When we take a closer look at the top 10 venues of the neighbourhoods in cluster 3, we can see that these are neighbourhoods that have a lot of focus on food, coffee shops; food culture is important to both cities, so it would stand to reason that the most similar neighbourhoods would have a lot of food and cafe venues.

# 5.0 Conclusion

In this project, we were comparing neighbourhoods in Downtown Toronto and Manhattan to determine their similarities in an attempt to provide insight to those who were interested in moving from Toronto to Manhattan.

While there are some neighbourhoods that share similarities (i.e. similar venues concerning food), it appears that the majority of neighbourhoods do not share much similarity between Toronto and Manhattan.

It should be noted that this analysis was done only taking into account nearby venues within 1km of each neighbourhood. Similarities in culture and environment can be determined in many ways, so do not solely rely on factors like venues to make your decision! 

Thank you for reading through this project.